In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# function which returns the number of occurnces of each element of an array
def count_occurrences(array):
    
    unique_elements, counts = np.unique(array, return_counts=True)
    
    occurrences = dict(zip(unique_elements, counts))
    
    return occurrences

def max(array):
    
    max_value = array[0]
    
    for i in range(1, len(array)):
        
        if array[i] > max_value:
            
            max_value = array[i]
    
    return max_value

# function which returns the local maximums positions and their time value
def local_max(time, position):
    
    max_pos = {}
    
    delta = int(len(position) / 200)
    i = 0
    j = 1
    while i < len(position):
        
        i_min = i - delta
        i_max = i + delta
        
        if (i_min < 0):    
            i_min = 0
        if (i_max > len(position)):
            i_max = len(position)
        
        if (position[i] == max(position[i_min:i_max])):
            
            max_pos[j] = [time[i], position[i]]
            j += 1
            
            i = i_max - 1
        
        i += 1
    
    return max_pos

# funtion which returns the matrix of all the possible periods of oscillations
def periods(maxima):
    
    p_osc = {}
    
    for key_1 in maxima:
        
        for key_2 in maxima:
        
            if (key_2 > key_1):
                
                p_osc[(key_1, key_2)] = ((maxima[key_2][0] - maxima[key_1][0]) / (key_2 - key_1))
    
    return p_osc

# functions which calculates the mean of the elements of a dictionary
def dic_mean(dic):
    
    n = 0
    sum = 0
    
    for key in dic:
        
        sum += dic[key]
        n += 1
    
    return (sum / n)



# analysis funtion

def analysis(path_equilibrio, path_oscillazioni, int_configurazione, plot_color, max_color):

    # import csv files for equilibrium position as DataFrames
    equilibrio = pd.read_csv(path_equilibrio, sep=';')
    # import csv files for oscillating positions as DataFrames
    oscillazione = pd.read_csv(path_oscillazioni, sep=';')

    # convert DatFrames of equilibrium positions to numpy arrays
    eq_time = equilibrio['time'].to_numpy()
    eq_pos = equilibrio['position'].to_numpy()

    # plot equilibrium positions
    plt.scatter(eq_time, eq_pos, marker='.', s=1, c=plot_color)
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel('x [m]')
    plt.title('Equilibrium position measured by photogate for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_equilibrio_"+str(int_configurazione)+".png", dpi=1200)
    plt.close()

    # plot histogran of equilibrium positions
    fig, axs= plt.subplots(1, 1, tight_layout = True)
    axs.hist(eq_pos, bins = 200, color=plot_color)
    plt.grid()
    plt.xlabel('x [m]')
    plt.ylabel('counts')
    plt.title('Histogram of equilibrium position for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_equilibrio_"+str(int_configurazione)+"_hist.png", dpi=1200)
    plt.close()

    # print number of equilibrium positions
    with open('analisi/smorzamento/output_equilibrio_'+str(int_configurazione)+'.log','w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        eq_pos_dic = count_occurrences(eq_pos)
        for pos in eq_pos_dic:
            print("x = ", pos, "\t n: ", eq_pos_dic[pos], file=textfile)

    # convert DatFrames of oscillating positions to numpy arrays
    osc_time = oscillazione['time'].to_numpy()
    osc_pos = oscillazione['position'].to_numpy()

    # print maximum positions
    with open('analisi/smorzamento/output_oscillazioni_'+str(int_configurazione)+'.log', 'w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        osc_max_dic = local_max(osc_time, osc_pos)

        for key in osc_max_dic:
            print(key, "\t x: ", osc_max_dic[key][1], "\t t: ", osc_max_dic[key][0], file=textfile)

    # evaluate period of oscillations
    with open('analisi/smorzamento/output_periodi_'+str(int_configurazione)+'.log', 'w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        p_osc = periods(osc_max_dic)

        for i in range(1, len(osc_max_dic)-1):
            for j in range(i+1, len(osc_max_dic)):
                print(i, "-", j, "\t T: ", p_osc[(i, j)], file=textfile)

        T = dic_mean(p_osc)
        print("\n\n\nThe mean period of oscillation is: ", T, file=textfile)

    # plot oscillating positions
    plt.scatter(osc_time, osc_pos, marker='.', s=1, c=plot_color)
    for key in osc_max_dic:
        plt.scatter(osc_max_dic[key][0], osc_max_dic[key][1], marker='.', s=3, c=max_color)
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel('x [m]')
    plt.title('Position measured by photogate for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_oscillazione_"+str(int_configurazione)+".png", dpi=1200)
    plt.close()


analysis('dati/smorzamento/equilibrio_1.csv', 'dati/smorzamento/oscillazione_1.csv', 1, 'blue', 'red')
analysis('dati/smorzamento/equilibrio_2.csv', 'dati/smorzamento/oscillazione_2.csv', 2, 'red', 'blue')
analysis('dati/smorzamento/equilibrio_3.csv', 'dati/smorzamento/oscillazione_3.csv', 3, 'green', 'red')